
# Project Apache Cassandra Data Modeling
## Project Overview
This notebook focuses on creating data models and tables in Apache Cassandra to address the following queries:
1. Retrieve artist, song title, and song's length for a specific `sessionId` and `itemInSession`.
2. Retrieve artist, song (sorted by `itemInSession`), and user (first and last name) for a specific `userId` and `sessionId`.
3. Retrieve all users who listened to a specific song.

Each query has been carefully modeled to optimize data partitioning, retrieval, and performance while adhering to Apache Cassandra's design principles.

The notebook is structured as follows:
- **Data Modeling and Table Creation**
- **Data Insertion**
- **Query Execution and Results**
- **Clean-up and Conclusion**


## Preambule to load time management features to the project

In [ ]:

import time

def timed_query(query, parameters=None):
    start = time.time()
    if parameters:
        rows = session.execute(query, parameters)
    else:
        rows = session.execute(query)
    end = time.time()
    print(f"Query executed in {end - start:.4f} seconds.")
    return rows


# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Creation of a Keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_history
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")


#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('music_history')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Data Modeling Approach of Query 1: Retrieve Song Details for a Session
The objective of **Query 1** is to retrieve the **artist name**, **song title**, and **song length** for a specific `sessionId` and `itemInSession`.

**Table Name**: `session_songs`

**Primary Key Components**:
- **Partition Key**: `sessionId`
- The `sessionId` groups all records for the same session into a single partition, ensuring efficient lookups.
- **Clustering Column**: `itemInSession`
- The `itemInSession` ensures uniqueness and natural order of rows within a partition.

#### **Why This Design?**

1. **Efficient Data Grouping**:
Using `sessionId` as the partition key ensures that all records related to the same session are stored together in a single partition. This design allows Cassandra to quickly locate and retrieve all the records associated with a specific session without scanning unrelated partitions.

2. **Uniqueness**:
Adding `itemInSession` as the clustering column guarantees that each record within the same session is uniquely identifiable. It also maintains the natural order of items within a session, ensuring accurate and organized retrieval of records.

3. **Optimized Query Execution**:
The query filters data based on `sessionId` and `itemInSession`, which aligns perfectly with the table's partition key and clustering column. This alignment eliminates the need for `ALLOW FILTERING`, avoiding costly full table scans and ensuring efficient query performance.

### **Final Table Design**
```sql
CREATE TABLE IF NOT EXISTS session_songs (
    sessionId INT,          -- Partition Key
itemInSession INT,      -- Clustering Column
artist TEXT,
       song TEXT,
            length FLOAT,
                   PRIMARY KEY (sessionId, itemInSession)
);



In [ ]:
## Query 1:  This query provide details on the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# Create table for Query 1
session.execute("""
    CREATE TABLE IF NOT EXISTS session_songs (
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        length FLOAT,
        PRIMARY KEY (sessionId, itemInSession)
    )
""")

In [ ]:
# Full Apache Cassandra code to insert the data in the target table
import csv

file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO session_songs (sessionId, itemInSession, artist, song, length)
            VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


#### SELECT statement to verify that the data have been inserted into each table

In [ ]:
## This SELECT statement help verifying that the data was appropriately entered into the table
query = """
    SELECT artist, song, length 
    FROM session_songs 
    WHERE sessionId = 338 AND itemInSession = 4
"""
rows = timed_query(query)
for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, Length: {row.length}")



### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS


### Data Modeling Strategy of Query 2 to Retrieve Sorted Song and User Details
#### Objective:
Fetch the **artist name**, **song title** (sorted by `itemInSession`), and **user details** (first and last name) for a specific `userId` and `sessionId`.

#### **How the Table is Designed**

##### Step 1: Partition Key
We use a **composite partition key** `(userId, sessionId)` to group records for a specific user session together. Grouping by `userId` and `sessionId` ensures that all data for the same user session is stored in one partition.

##### Step 2: Clustering Column
We use `itemInSession` as the clustering column. It **sorts** the records within the partition in ascending order of `itemInSession`.

#### **Query Flow**

1. Filter by `userId` and `sessionId` → Locate the partition.
2. Retrieve rows → Sorted naturally by `itemInSession`.

#### **Advantages**

1. **Logical Grouping**: Data is grouped by `userId` and `sessionId`.
2. **Sorting**: Sorting by `itemInSession` is built into the clustering column.
3. **No ALLOW FILTERING**: Query is efficient and avoids unnecessary scans.

#### **Table Definition**
```sql
CREATE TABLE IF NOT EXISTS user_sessions (
    userId INT,               -- Partition Key Component
sessionId INT,            -- Partition Key Component
itemInSession INT,        -- Clustering Column
artist TEXT,
       song TEXT,
            firstName TEXT,
                      lastName TEXT,
                               PRIMARY KEY ((userId, sessionId), itemInSession)
);


In [ ]:
## Query 2: This query provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# Create table for Query 2
session.execute("""
    CREATE TABLE IF NOT EXISTS user_sessions (
        userId INT,
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
""")


In [ ]:
# Insertion of data into user_sessions table
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, song, firstName, lastName)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))



### Query Execution
This query retrieves the required data based on the defined partition key and clustering columns, without using `ALLOW FILTERING`.


In [ ]:
# Verifying the data for userId = 10 and sessionId = 182
query = """
    SELECT artist, song, firstName, lastName 
    FROM user_sessions 
    WHERE userId = 10 AND sessionId = 182
"""
rows = timed_query(query)
for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, User: {row.firstname} {row.lastname}")




### Data Modeling Roadmap for Query 3 to Extract All Users for a Specific Song
### Objective:
Retrieve the **first and last names** of all users who listened to a particular song.

### **Table Design at a Glance**

- **Table Name**: `song_listeners`
- **Partition Key**: `song`
- Ensures that all listeners of the same song are stored together in one partition.
- **Clustering Column**: `userId`
- Ensures each user appears **only once** in the partition, avoiding duplicates.

### **Why is this Table Design Effective?**

- **Efficient Data Retrieval**:
By partitioning data using `song`, we can quickly retrieve all records for a specific song without scanning unrelated partitions.

- **Unique User Identification**:
The clustering column `userId` prevents duplicate entries for users within the same song partition.

- **Optimal Query Execution**:
The query filters on `song`, which aligns perfectly with the partition key. This avoids the need for `ALLOW FILTERING` and ensures fast performance.

### **Table Definition**
```sql
CREATE TABLE IF NOT EXISTS song_listeners (
    song TEXT,                -- Partition Key
userId INT,               -- Clustering Column
firstName TEXT,
          lastName TEXT,
                   PRIMARY KEY (song, userId)
);


In [ ]:
# Create table for Query 3
session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners (
        song TEXT,
        userId INT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY (song, userId)
    )
""")


In [ ]:
# Insertion of the data into the song_listeners table
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO song_listeners (song, userId, firstName, lastName)
            VALUES (%s, %s, %s, %s)
        """
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))



### Query Execution
This query retrieves the required data based on the defined partition key and clustering columns, without using `ALLOW FILTERING`.


In [ ]:
## Query 3 aims at givin every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
query = """
    SELECT firstName, lastName 
    FROM song_listeners 
    WHERE song = 'All Hands Against His Own'
"""
rows = timed_query(query)
for row in rows:
    print(f"User: {row.firstname} {row.lastname}")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Execute the query
query = "SELECT song, length FROM session_songs"
rows = session.execute(query)

# Convert rows to a list of tuples
rows_list = list(rows)

# Create DataFrame
df = pd.DataFrame(rows_list, columns=['song', 'length'])

# Ensure 'length' is numeric and convert from seconds to minutes
df['length'] = pd.to_numeric(df['length'], errors='coerce') / 60  # Convert to minutes

# Remove duplicates based on 'song' and keep the minimum duration
df = df.drop_duplicates(subset='song', keep='first')

# Sort by 'length' in ascending order
df_sorted = df.sort_values(by='length', ascending=False)

# Select the top 10 songs (from shortest to longest duration)
top_songs_ascending = df_sorted.head(10)

# Plot the results
plt.figure(figsize=(10, 10))
plt.bar(top_songs_ascending['song'], top_songs_ascending['length'])
plt.xticks(rotation=45, ha="right")
plt.title('Top 10 Songs by Length')
plt.xlabel('Song')
plt.ylabel('Length (minutes)', )
plt.tight_layout()
plt.show()


In [ ]:
### Drop the tables before closing out the sessions

In [29]:
## Dropping the tables before closing out the sessions
session.execute("DROP TABLE IF EXISTS session_songs")
session.execute("DROP TABLE IF EXISTS user_sessions")
session.execute("DROP TABLE IF EXISTS song_listeners")
print("Tables dropped successfully.")


Tables dropped successfully.


### Close the session and cluster connection¶

In [30]:
# Close the session and cluster connection
session.shutdown()
cluster.shutdown()
print("Cassandra session and cluster connection closed.")


Cassandra session and cluster connection closed.



## Clean-up and Conclusion
The tables have been dropped, and the Cassandra session and cluster connection have been closed to ensure resource cleanup.

This notebook demonstrates how to correctly model data in Apache Cassandra to answer specific queries, optimize data partitioning, and avoid `ALLOW FILTERING`. The outputs have been presented in a clear and readable format using Pandas DataFrames.

**Best Practices Followed**:
- Logical table and column naming.
- Efficient use of partition keys and clustering columns.
- Avoidance of `ALLOW FILTERING`.
- Modular and clean notebook structure.
